In [1]:
import pandas as pd
import numpy as np
import json
import pprint
import requests
from sqlalchemy import create_engine, inspect
from sqlalchemy import Column, Integer, String, Float

In [2]:
#USER_01_MAPPING = 'resources/raw_data_next_gen/20200329222159_FieldDataHarvest_Users.csv'
PROJECT_01_MAPPING = 'resources/raw_data_next_gen/admin_projects.csv'
ISSUE_01_MAPPING = 'resources/raw_data_next_gen/issues_issues.csv'

##############

USER_02_MAPPING = 'resources/raw_data_old_gen/20200330085907_FieldDataHarvest_Users.csv'
PROJECT_02_MAPPING = 'resources/raw_data_old_gen/20200330085907_FieldDataHarvest_Projects.csv'
ISSUE_02_MAPPING = 'resources/raw_data_old_gen/20200330085907_FieldDataHarvest_Issues.csv'

In [3]:
# U1_DF = pd.read_csv(USER_01_MAPPING, encoding='utf-8')
P1_DF = pd.read_csv(PROJECT_01_MAPPING, encoding='utf-8')
I1_DF = pd.read_csv(ISSUE_01_MAPPING, encoding='latin1')

U2_DF = pd.read_csv(USER_02_MAPPING, encoding='utf-8')
P2_DF = pd.read_csv(PROJECT_02_MAPPING, encoding='utf-8')
I2_DF = pd.read_csv(ISSUE_02_MAPPING, encoding='latin1')


C:\Users\zigot\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (4,13,14,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#I1_DF.count()

In [ ]:
#I2_DF.count()

In [ ]:
#U2_DF.count()

In [ ]:
#P2_DF.count()

In [ ]:
#P2_DF.head()

In [ ]:
test = I2_DF['project_id'].unique()

In [ ]:
len(test)

In [ ]:
PC_s=PC.sort_values(by=['project_id'])

In [ ]:
#PC_s.head(50)

In [11]:
MERGED_OLD = pd.merge(P2_DF, I2_DF, on='project_id', how='inner')


In [ ]:
#MERGED_DF.count()

In [17]:
P1_DF.rename(columns = {"id":"project_id"})


,project_id,bim360_account_id,name,start_date,end_date,type,value,currency,status,job_number,...,city,state_or_province,postal_code,country,timezone,construction_type,contract_type,business_unit_id,last_sign_in,created_at
0,4fb9e8e5-f9a7-4a04-b33c-38f2227bf75d,49df69ab-c266-4d49-951c-031b16f7a31f,Amanda,2012-05-01 00:00:00,2115-01-01 00:00:00,Hospitals & Health Treatment Buildings,750000.0,USD,archived,NaN,...,St Louis,MO,63131,US,America/Chicago,NaN,NaN,NaN,NaN,2015-08-27 03:02:51.944348
1,e5ed857f-23f1-4a8c-a808-9cbfa0fe0648,49df69ab-c266-4d49-951c-031b16f7a31f,10100726-UCMC-GARAGE,2016-08-23 00:00:00,2021-08-24 00:00:00,Office,0.0,USD,archived,NaN,...,NaN,NaN,NaN,US,NaN,NaN,NaN,NaN,NaN,2016-08-24 01:51:54.201861
2,9d602269-4385-4d46-9d87-ee1374608e23,49df69ab-c266-4d49-951c-031b16f7a31f,10100726-UCMC,2016-08-23 00:00:00,2021-08-24 00:00:00,Office,0.0,USD,archived,NaN,...,NaN,NaN,NaN,US,NaN,NaN,NaN,NaN,NaN,2016-08-24 01:51:52.866017
3,003d373a-2b90-45f8-8a0e-862868dfdc0b,49df69ab-c266-4d49-951c-031b16f7a31f,BIM Test Ground,1915-01-01 00:00:00,2115-01-01 00:00:00,default,0.0,USD,archived,NaN,...,NaN,NaN,63114,US,America/Chicago,NaN,NaN,NaN,NaN,2015-08-27 03:03:02.549974
4,9fcfaa9b-b11e-47d9-acbb-a8a8ddf1f7ac,49df69ab-c266-4d49-951c-031b16f7a31f,"Thompson Coburn (ORD), Phase 3",2011-08-28 00:00:00,2115-01-01 00:00:00,Office & Bank Buildings,0.0,USD,archived,NaN,...,Chicago,IL,60603,US,America/Chicago,NaN,NaN,NaN,NaN,2015-08-27 03:03:27.170199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770,06280667-18de-43c2-b8e2-a00d341c3d9b,49df69ab-c266-4d49-951c-031b16f7a31f,10104287-Pepsi TI North Park,2019-06-17 00:00:00,2020-06-15 00:00:00,Warehouse (non-manufacturing),3763709.0,USD,active,NaN,...,St. Louis,MO,NaN,US,America/New_York,NaN,NaN,NaN,2020-03-05 14:16:38.456,2019-06-17 21:36:13.316405
771,88196ace-66d3-492a-8270-77938d3501aa,49df69ab-c266-4d49-951c-031b16f7a31f,10104224-Chewy,2018-10-01 00:00:00,2019-08-01 00:00:00,Manufacturing / Factory,26855095.0,USD,active,10104224,...,Dayton,OH,45377,US,America/New_York,New Construction,Design-Bid,869871b3-ac33-4a98-ab47-0ede0ba28d1c,2020-01-10 02:35:05.654,2019-02-03 22:54:27.509075
772,ad7bf4f4-2dc0-4e4c-9ef2-513c72d6209a,49df69ab-c266-4d49-951c-031b16f7a31f,10104220 - Edge at West,2019-04-12 00:00:00,2020-03-01 00:00:00,Office,17500000.0,USD,active,10104220,...,NaN,NaN,NaN,US,America/New_York,New Construction,Design-Bid,c90a5237-c3d5-11e2-a4be-02cc3afa57bd,2020-03-26 17:05:49.942,2019-04-12 21:11:34.227245
773,f69194b9-8fad-4cba-a724-091853457fb3,49df69ab-c266-4d49-951c-031b16f7a31f,10104201 - Harbor Freight - Chicago,2019-10-01 00:00:00,2020-12-31 00:00:00,Warehouse (non-manufacturing),100000000.0,USD,active,NaN,...,NaN,NaN,NaN,US,America/New_York,NaN,NaN,NaN,2020-03-19 18:05:34.231,2019-09-26 18:37:42.92614


In [ ]:
#P2_DF.head()

In [18]:
I1_DF.rename(columns = {"bim360_project_id":"project_id"})

,issue_id,bim360_account_id,project_id,display_id,title,description,type_id,subtype_id,status,assignee_id,...,response_by,response_at,opened_by,opened_at,closed_by,closed_at,created_by,created_at,updated_by,updated_at
0,878438f4-f7a2-4a5d-80aa-739b66a63d38,49df69ab-c266-4d49-951c-031b16f7a31f,14c8029b-1daf-4fdc-ae19-a5932d354db3,218,DISCUSSION - Visit with the worker(s) and disc...,NaN,d880b3ee-fafa-4892-9ec0-79d590e1aa11,10681c25-127c-4de8-b18b-8ab3b07492e8,void,NaN,...,NaN,NaN,WBKJREULUNNP,2019-11-12 20:46:54.510863,WBKJREULUNNP,2019-11-12 21:08:35.46027,WBKJREULUNNP,2019-11-12 20:46:54.526565,WBKJREULUNNP,2019-11-12 21:08:35.46911
1,0032889e-94c2-4066-ba36-5ee999fee93b,49df69ab-c266-4d49-951c-031b16f7a31f,72ee4856-b251-4ebe-b996-3b2daa7f1408,103,WestG,I was walking the site and noticed a piece of ...,7bc5f23d-db09-4ddb-9be8-43f9c16fc42a,f47dba9a-569b-4c83-a63d-9eda3c11031c,closed,JWSYMWGJ46FU,...,JWSYMWGJ46FU,2019-03-21 21:47:50.560542,JWSYMWGJ46FU,2019-03-21 21:47:29.161767,APQNL5V27TY8,2019-03-26 11:17:41.227531,JWSYMWGJ46FU,2019-03-21 21:47:29.17616,APQNL5V27TY8,2019-03-26 11:17:41.234357
2,edb626cc-33d4-4bcf-b4de-a12bfbc24f1f,49df69ab-c266-4d49-951c-031b16f7a31f,bd25b19e-8398-4c4c-ae3f-5c60a66b78b9,678,Cut in wall riser pipe,Trim in wall piping so that it is not pushing ...,0c8ebb30-3e79-4186-9121-ee6a6618a0db,83f98667-e1c4-4ffd-b8e1-ff7a650a8aa6,closed,8MVYQWA3RXDL,...,LKYQRW9DSKK2,2019-07-02 16:56:12.234437,8MVYQWA3RXDL,2019-06-26 18:19:43.543099,8MVYQWA3RXDL,2019-07-17 16:05:00.794415,8MVYQWA3RXDL,2019-06-26 18:19:43.555365,8MVYQWA3RXDL,2019-07-17 16:05:00.804621
3,f4f4f566-eac5-4362-80f6-db751678aa06,49df69ab-c266-4d49-951c-031b16f7a31f,f3b1fc0d-f9bb-46c1-a9c5-dc8255da5909,64,Paint panel,NaN,bb99b695-2633-4bd1-8c49-ea3790089377,964b6fed-46b3-43bf-8ca0-0495d9326dff,work_completed,55979794,...,NaN,NaN,AFPSLVB4KRQT,2019-03-21 14:40:02.993482,NaN,NaN,AFPSLVB4KRQT,2019-03-21 14:40:03.004759,R6EMKU6UPBXQ,2019-11-02 18:40:23.947218
4,3c519fa1-ee33-4a76-bb55-0a74a177a0e8,49df69ab-c266-4d49-951c-031b16f7a31f,d2310a2c-8320-49b5-b43e-97260b67c88d,143,"LADDERS / SCAFFOLDING - All Ladders type 1A, s...",Found ladder without stickers,7d97ff16-23c5-40e4-9386-d78237c4c1d9,96731678-cb60-42f6-9bc3-492cb0708614,closed,3URYK5ARM6DF,...,3URYK5ARM6DF,2019-03-20 16:17:08.170965,3URYK5ARM6DF,2019-03-20 16:16:37.055898,3URYK5ARM6DF,2019-03-20 16:17:27.94132,3URYK5ARM6DF,2019-03-20 16:16:37.072966,3URYK5ARM6DF,2019-03-20 16:17:27.947626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59079,2ae6dc53-6182-40c0-adff-edfc6b77aa9c,49df69ab-c266-4d49-951c-031b16f7a31f,4c3d6388-16b9-4a0a-bdca-16bd7f41074c,667,Clean corner guard,NaN,34590997-9334-4448-8260-f7b1cfd1e09c,c303f2eb-3d12-4809-a05f-e2e71aa48593,open,NaN,...,NaN,NaN,KUYHKGGRU8ZC,2020-03-26 20:22:26.152652,NaN,NaN,KUYHKGGRU8ZC,2020-03-26 20:22:26.16398,KUYHKGGRU8ZC,2020-03-26 20:22:32.021843
59080,d0a1dbde-87c9-4718-aaba-465ea5462da0,49df69ab-c266-4d49-951c-031b16f7a31f,4c3d6388-16b9-4a0a-bdca-16bd7f41074c,670,Replace tile,NaN,34590997-9334-4448-8260-f7b1cfd1e09c,7b8adf88-16e0-4411-9b76-6b684e7647a3,open,NaN,...,NaN,NaN,C3ZEL7MRCFGM,2020-03-26 20:24:37.364716,NaN,NaN,C3ZEL7MRCFGM,2020-03-26 20:24:37.380301,C3ZEL7MRCFGM,2020-03-26 20:24:41.532582
59081,3415eaa1-b475-4e09-bb05-df390e99de62,49df69ab-c266-4d49-951c-031b16f7a31f,4c3d6388-16b9-4a0a-bdca-16bd7f41074c,671,Replace grid,NaN,34590997-9334-4448-8260-f7b1cfd1e09c,7b8adf88-16e0-4411-9b76-6b684e7647a3,open,NaN,...,NaN,NaN,C3ZEL7MRCFGM,2020-03-26 20:26:00.869438,NaN,NaN,C3ZEL7MRCFGM,2020-03-26 20:26:00.885018,C3ZEL7MRCFGM,2020-03-26 20:26:04.090792
59082,643ecdab-7799-45af-91b9-dd25c9f2e789,49df69ab-c266-4d49-951c-031b16f7a31f,4c3d6388-16b9-4a0a-bdca-16bd7f41074c,672,Caulk and adjust sill extension,NaN,34590997-9334-4448-8260-f7b1cfd1e09c,7b8adf88-16e0-4411-9b76-6b684e7647a3,open,NaN,...,NaN,NaN,C3ZEL7MRCFGM,2020-03-26 20:27:36.248131,NaN,NaN,C3ZEL7MRCFGM,2020-03-26 20:27:36.259992,C3ZEL7MRCFGM,2020-

In [ ]:
#I2_DF.head()

In [9]:
#MERGED_NEW = pd.merge(P1_DF, I1_DF, on='project_id', how='inner')

In [42]:
MERGED_OLD.to_csv('old_clean.csv')

In [38]:
P1_DF.rename(columns = {"id" : "project_id"}, inplace=True)

In [23]:
I1_DF.rename(columns = {"bim360_project_id" : "project_id"}, inplace=True)

In [39]:
I1_DF.dtypes

issue_id                  object
bim360_account_id         object
project_id                object
display_id                 int64
title                     object
description               object
type_id                   object
subtype_id                object
status                    object
assignee_id               object
assignee_type             object
due_date                  object
location_id               object
location_details          object
linked_document_urn       object
owner_id                  object
root_cause_id             object
root_cause_category_id    object
response                  object
response_by               object
response_at               object
opened_by                 object
opened_at                 object
closed_by                 object
closed_at                 object
created_by                object
created_at                object
updated_by                object
updated_at                object
dtype: object

In [37]:
P1_DF.dtypes

id                    object
bim360_account_id     object
name                  object
start_date            object
end_date              object
type                  object
value                float64
currency              object
status                object
job_number            object
address_line1         object
address_line2         object
city                  object
state_or_province     object
postal_code           object
country               object
timezone              object
construction_type     object
contract_type         object
business_unit_id      object
last_sign_in          object
created_at            object
dtype: object

In [40]:
MERGED_NEW = pd.merge(P1_DF, I1_DF, on ='project_id', how='inner')

In [41]:
MERGED_NEW.count()

project_id                59084
bim360_account_id_x       59084
name                      59084
start_date                58974
end_date                  58974
type                      59084
value                     57593
currency                  59084
status_x                  59084
job_number                57085
address_line1             42603
address_line2               883
city                      42841
state_or_province         42592
postal_code               42401
country                   59084
timezone                  59084
construction_type         39531
contract_type             39715
business_unit_id          57294
last_sign_in              59084
created_at_x              59084
issue_id                  59084
bim360_account_id_y       59084
display_id                59084
title                     59084
description               23200
type_id                   59084
subtype_id                59084
status_y                  59084
assignee_id               52446
assignee

In [43]:
MERGED_NEW.to_csv('new_clean.csv')

In [44]:
P1_DF.head()

,project_id,bim360_account_id,name,start_date,end_date,type,value,currency,status,job_number,...,city,state_or_province,postal_code,country,timezone,construction_type,contract_type,business_unit_id,last_sign_in,created_at
0,4fb9e8e5-f9a7-4a04-b33c-38f2227bf75d,49df69ab-c266-4d49-951c-031b16f7a31f,Amanda,2012-05-01 00:00:00,2115-01-01 00:00:00,Hospitals & Health Treatment Buildings,750000.0,USD,archived,NaN,...,St Louis,MO,63131,US,America/Chicago,NaN,NaN,NaN,NaN,2015-08-27 03:02:51.944348
1,e5ed857f-23f1-4a8c-a808-9cbfa0fe0648,49df69ab-c266-4d49-951c-031b16f7a31f,10100726-UCMC-GARAGE,2016-08-23 00:00:00,2021-08-24 00:00:00,Office,0.0,USD,archived,NaN,...,NaN,NaN,NaN,US,NaN,NaN,NaN,NaN,NaN,2016-08-24 01:51:54.201861
2,9d602269-4385-4d46-9d87-ee1374608e23,49df69ab-c266-4d49-951c-031b16f7a31f,10100726-UCMC,2016-08-23 00:00:00,2021-08-24 00:00:00,Office,0.0,USD,archived,NaN,...,NaN,NaN,NaN,US,NaN,NaN,NaN,NaN,NaN,2016-08-24 01:51:52.866017
3,003d373a-2b90-45f8-8a0e-862868dfdc0b,49df69ab-c266-4d49-951c-031b16f7a31f,BIM Test Ground,1915-01-01 00:00:00,2115-01-01 00:00:00,default,0.0,USD,archived,NaN,...,NaN,NaN,63114,US,America/Chicago,NaN,NaN,NaN,NaN,2015-08-27 03:03:02.549974
4,9fcfaa9b-b11e-47d9-acbb-a8a8ddf1f7ac,49df69ab-c266-4d49-951c-031b16f7a31f,"Thompson Coburn (ORD), Phase 3",2011-08-28 00:00:00,2115-01-01 00:00:00,Office & Bank Buildings,0.0,USD,archived,NaN,...,Chicago,IL,60603,US,America/Chicago,NaN,NaN,NaN,NaN,2015-08-27 03:03:27.170199


In [45]:
P1_DF.count()

project_id           775
bim360_account_id    775
name                 775
start_date           666
end_date             662
type                 775
value                616
currency             775
status               775
job_number           355
address_line1        300
address_line2         25
city                 338
state_or_province    340
postal_code          384
country              775
timezone             741
construction_type    348
contract_type        269
business_unit_id     408
last_sign_in         500
created_at           775
dtype: int64

In [51]:
MERGED_OLD.rename(columns = {"project_id" : "project_id_classic"}, inplace=True)
MERGED_OLD.rename(columns = {"hq_identifier_x" : "project_id"}, inplace=True)

In [52]:
MERGED_OLD_NEXT = pd.merge(MERGED_OLD, P1_DF, on='project_id', how='inner')

In [53]:
MERGED_OLD_NEXT.head()

,project_id,business_unit_x,project_id_classic,name_x,status_x,files,issues,checklists,equipment,tasks,...,city,state_or_province,postal_code,country,timezone,construction_type,contract_type,business_unit_id,last_sign_in,created_at_y
0,01fa4f3a-9969-4069-a5b3-72fb630a0143,"Clayco>Distribution, Manufacturing, Process, F...",6fbfddc0-b8bb-4b27-be7f-751a21c701d0,Gestamp,active,177,476,0,0,0,...,McCalla,AL,35111,US,America/Chicago,New Construction,Design-Bid-Build,39f43f7d-1eee-11e3-89a6-02cc3afa57bd,2020-01-20 17:42:48,2016-08-09 16:41:14.956157
1,01fa4f3a-9969-4069-a5b3-72fb630a0143,"Clayco>Distribution, Manufacturing, Process, F...",6fbfddc0-b8bb-4b27-be7f-751a21c701d0,Gestamp,active,177,476,0,0,0,...,McCalla,AL,35111,US,America/Chicago,New Construction,Design-Bid-Build,39f43f7d-1eee-11e3-89a6-02cc3afa57bd,2020-01-20 17:42:48,2016-08-09 16:41:14.956157
2,01fa4f3a-9969-4069-a5b3-72fb630a0143,"Clayco>Distribution, Manufacturing, Process, F...",6fbfddc0-b8bb-4b27-be7f-751a21c701d0,Gestamp,active,177,476,0,0,0,...,McCalla,AL,35111,US,America/Chicago,New Construction,Design-Bid-Build,39f43f7d-1eee-11e3-89a6-02cc3afa57bd,2020-01-20 17:42:48,2016-08-09 16:41:14.956157
3,01fa4f3a-9969-4069-a5b3-72fb630a0143,"Clayco>Distribution, Manufacturing, Process, F...",6fbfddc0-b8bb-4b27-be7f-751a21c701d0,Gestamp,active,177,476,0,0,0,...,McCalla,AL,35111,US,America/Chicago,New Construction,Design-Bid-Build,39f43f7d-1eee-11e3-89a6-02cc3afa57bd,2020-01-20 17:42:48,2016-08-09 16:41:14.956157
4,01fa4f3a-9969-4069-a5b3-72fb630a0143,"Clayco>Distribution, Manufacturing, Process, F...",6fbfddc0-b8bb-4b27-be7f-751a21c701d0,Gestamp,active,177,476,0,0,0,...,McCalla,AL,35111,US,America/Chicago,New Construction,Design-Bid-Build,39f43f7d-1eee-11e3-89a6-02cc3afa57bd,2020-01-20 17:42:48,2016-08-09 16:41:14.956157


In [54]:
#MERGED_OLD.head()

In [50]:
#P2_DF.head()

In [56]:
#MERGED_OLD_NEXT.count()

In [57]:
MERGED_OLD_NEXT.to_csv('old_next_clean.csv')

In [66]:
### FINAL MERGE PROJECT ISSUES
OLD_MAPPING = 'old_next_clean.csv'
NEW_MAPPING = 'new_clean.csv'

In [67]:
OLD_DF = pd.read_csv(OLD_MAPPING, encoding='utf-8', low_memory=False)
NEW_DF = pd.read_csv(NEW_MAPPING, encoding='utf-8', low_memory=False)

In [72]:
OLD_DF.count()
# Working_OLD_DataFrame
W_OLD_DF = OLD_DF[['project_id', 'business_unit_x','name_x','issues','users','issue_id','description','issue_type','status_x','company_assigned','job_number',]].copy()

In [73]:
W_OLD_DF.head()

,project_id,business_unit_x,name_x,issues,users,issue_id,description,issue_type,status_x,company_assigned,job_number
0,01fa4f3a-9969-4069-a5b3-72fb630a0143,"Clayco>Distribution, Manufacturing, Process, F...",Gestamp,476,40,00bc0784-ed90-41fe-97d2-a71831f0e1dd,Remove nails around dock angles.,Punch List : Pre-Punch List,active,Concrete Strategies LLC,102601
1,01fa4f3a-9969-4069-a5b3-72fb630a0143,"Clayco>Distribution, Manufacturing, Process, F...",Gestamp,476,40,00d3a27e-8127-4dec-ba44-74da29274ac8,"Grind in a 3/4"" chamfer at all exterior corner...",Punch List : Pre-Punch List,active,Concrete Strategies LLC,102601
2,01fa4f3a-9969-4069-a5b3-72fb630a0143,"Clayco>Distribution, Manufacturing, Process, F...",Gestamp,476,40,00dfc82b-7220-4345-93ea-d0ff4ba06d03,Install missing switch and data outlets,Punch List : Architect Punch List,active,"Ace Electric, Inc.",102601
3,01fa4f3a-9969-4069-a5b3-72fb630a0143,"Clayco>Distribution, Manufacturing, Process, F...",Gestamp,476,40,01a36881-f79f-4308-9fa1-55b48f7c58f2,Install cover plate on light switch,Punch List : Architect Punch List,active,"Ace Electric, Inc.",102601
4,01fa4f3a-9969-4069-a5b3-72fb630a0143,"Clayco>Distribution, Manufacturing, Process, F...",Gestamp,476,40,01cebbe8-8a06-489f-a724-bb1b94844b0b,Install base,Punch List : Architect Punch List,active,Spectra Contract Flooring,102601


In [64]:
NEW_DF.count()

Unnamed: 0                59084
project_id                59084
bim360_account_id_x       59084
name                      59084
start_date                58974
end_date                  58974
type                      59084
value                     57593
currency                  59084
status_x                  59084
job_number                57085
address_line1             42603
address_line2               883
city                      42841
state_or_province         42592
postal_code               42401
country                   59084
timezone                  59084
construction_type         39531
contract_type             39715
business_unit_id          57294
last_sign_in              59084
created_at_x              59084
issue_id                  59084
bim360_account_id_y       59084
display_id                59084
title                     59084
description               23200
type_id                   59084
subtype_id                59084
status_y                  59084
assignee